In [50]:
# =========================================
# 📦 Dependências e Bibliotecas
# =========================================
!pip install tensorflow numpy pandas matplotlib sklearn openpyxl

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from google.colab import files
from itertools import product

  Using cached sklearn-0.0.post12.tar.gz (2.6 kB)
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [51]:
# 📁 Carregar Dados
# =========================================
uploaded = files.upload()
df_original = pd.read_excel(next(iter(uploaded)))



Saving OrganizationType_Count_v4_semajust.xlsx to OrganizationType_Count_v4_semajust.xlsx


In [52]:
# Conferir as colunas disponíveis

display(df_original.sample(10))

,Date Breach,BSF,BSO,BSR,EDU,GOV,MED,NGO,UNKN,Total Geral
191,01/10/2017,35.0,48.0,40.0,4.0,12.0,34.0,2.0,68.0,243
34,01/02/2017,29.0,44.0,36.0,33.0,5.0,61.0,10.0,164.0,382
125,01/07/2010,1.0,2.0,NaN,NaN,NaN,2.0,NaN,1.0,6
0,01/01/1990,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1
214,01/11/2021,44.0,132.0,81.0,34.0,12.0,101.0,23.0,166.0,593
53,01/03/2016,26.0,58.0,31.0,14.0,5.0,63.0,3.0,146.0,346
41,01/02/2024,75.0,91.0,19.0,21.0,8.0,193.0,7.0,79.0,493
131,01/07/2016,27.0,20.0,33.0,3.0,NaN,47.0,3.0,43.0,176
130,01/07/2015,13.0,6.0,20.0,1.0,2.0,15.0,NaN,37.0,94
219,01/12/2007,1.0,2.0,2.0,NaN,NaN,2.0,NaN,1.0,8


In [53]:
soma_total = df_original["Total Geral"].sum()
print(f"Soma da coluna 'Total Geral': {soma_total}")


Soma da coluna 'Total Geral': 40109


In [54]:
# Faz uma cópia do DataFrame original para evitar alterações permanentes
df = df_original.copy()

# Função para ajustar datas incompletas (ano ou ano/mês) e tratá-las como datas válidas
def ajustar_datas(data_str):
    try:
        if len(data_str) == 4:  # Somente ano (ex: '2005') → assume '01-01'
            return pd.Timestamp(data_str + '-01-01')
        elif len(data_str) == 7:  # Ano e mês (ex: '2005-09') → assume dia '01'
            return pd.Timestamp(data_str + '-01')
        else:  # Data completa (ex: '2005-09-22')
            return pd.Timestamp(data_str)
    except:
        return pd.NaT  # Retorna valor nulo se a conversão falhar

# Ajusta as datas, se necessário
df['Date Breach'] = df['Date Breach'].apply(ajustar_datas)
df = df.dropna(subset=['Date Breach'])

# Define a coluna de data como índice do DataFrame
df.set_index('Date Breach', inplace=True)
df.index = pd.to_datetime(df.index)

# Filtra para manter apenas dados antes de 2024 (se necessário)
df = df[df.index < pd.to_datetime('2024-01-01')]

# Verifica 50 registros aleatórios da base mensal consolidada
display(df.sample(50))

# Soma total da coluna 'BSF' (ou qualquer outra coluna que deseje)
soma_total = df["BSF"].sum()
print(f"Soma da coluna 'BSF': {soma_total}")

# Salva o DataFrame consolidado em um CSV
df.to_csv("dados_agregados_mensal.csv")
print("📁 Arquivo 'dados_agregados_mensal.csv' salvo com sucesso!")


,BSF,BSO,BSR,EDU,GOV,MED,NGO,UNKN,Total Geral
Date Breach,,,,,,,,,
2000-01-01,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,1
2014-01-12,14.0,12.0,20.0,3.0,NaN,36.0,4.0,42.0,131
2018-01-05,27.0,49.0,43.0,17.0,7.0,98.0,3.0,98.0,342
2017-01-01,42.0,43.0,41.0,27.0,9.0,48.0,4.0,139.0,353
2013-01-10,6.0,8.0,7.0,9.0,NaN,20.0,5.0,17.0,72
2011-01-10,3.0,2.0,2.0,1.0,NaN,7.0,NaN,5.0,20
2014-01-10,9.0,6.0,23.0,3.0,NaN,18.0,NaN,44.0,103
2008-01-07,1.0,3.0,NaN,NaN,NaN,1.0,NaN,NaN,5
2013-01-02,9.0,9.0,6.0,5.0,1.0,13.0,NaN,10.0,53


Soma da coluna 'BSF': 4331.0
📁 Arquivo 'dados_agregados_mensal.csv' salvo com sucesso!


In [57]:
# Lista completa dos setores a serem analisados
setores = ['BSF']#, 'BSO', 'BSR', 'EDU', 'GOV', 'MED', 'NGO', 'Total Geral','UNKN']

# Lista para armazenar os resultados de cada combinação
resultados = []

from itertools import product

# Definição dos hiperparâmetros para Grid Search
look_back_values = [6, 12, 18]
epochs_values = [100, 200, 300]
batch_sizes = [16, 32, 64]

# Gera todas as combinações possíveis
param_grid = list(product(look_back_values, epochs_values, batch_sizes))

# Função para transformar a série temporal em formato para LSTM
def create_dataset(dataset, look_back=6):
    X, y = [], []
    for i in range(len(dataset) - look_back):
        X.append(dataset[i:(i + look_back)])
        y.append(dataset[i + look_back])
    return np.array(X), np.array(y)


# LOOP por setor e combinação de hiperparâmetros
for setor in setores:
    print(f"\n🚩 Iniciando treinamento para setor: {setor}")
    df = df_original.copy()

    # Ajuste de datas
    def ajustar_datas(data_str):
        try:
            if len(data_str) == 4:
                return pd.Timestamp(data_str + '-01-01')
            elif len(data_str) == 7:
                return pd.Timestamp(data_str + '-01')
            else:
                return pd.Timestamp(data_str)
        except:
            return pd.NaT

    # Conversão de datas, se necessário
    df['Date Breach'] = df['Date Breach'].apply(ajustar_datas)
    df = df.dropna(subset=['Date Breach'])
    df.set_index('Date Breach', inplace=True)
    df.index = pd.to_datetime(df.index)

    # Filtro para manter apenas dados antes de 2024
    df = df[df.index < pd.to_datetime('2024-01-01')]

    # ➤ NÃO há resample aqui, pois os dados já estão agregados mensalmente
    serie = df[[setor]].fillna(0)

    # Normalização
    scaler = MinMaxScaler()
    serie_scaled = scaler.fit_transform(serie)

    for look_back, epochs, batch_size in param_grid:
        print(f"\n🛠️  Parâmetros: look_back={look_back}, epochs={epochs}, batch_size={batch_size}")

        X, y = create_dataset(serie_scaled, look_back)
        X_train, X_test = X[:-24], X[-24:]
        y_train, y_test = y[:-24], y[-24:]

        X_train = X_train.reshape(X_train.shape[0], look_back, 1)
        X_test = X_test.reshape(X_test.shape[0], look_back, 1)

        # Modelo LSTM
        model = Sequential()
        model.add(LSTM(50, activation='tanh', input_shape=(look_back, 1)))
        model.add(Dense(1))
        model.compile(optimizer='adam', loss='mse')

        model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=0)

        # Previsão
        y_pred_scaled = model.predict(X_test)
        y_pred = scaler.inverse_transform(y_pred_scaled)
        y_pred = np.clip(y_pred, 0, None)  # remove negativos

        y_real = scaler.inverse_transform(y_test)

        # Métricas
        mae = mean_absolute_error(y_real, y_pred)
        rmse = np.sqrt(mean_squared_error(y_real, y_pred))
        mape = np.mean(np.abs((y_real[y_real != 0] - y_pred[y_real != 0]) / y_real[y_real != 0])) * 100

        print(f"📉 Métricas de avaliação para setor {setor}:")
        print(f"MAE: {mae:.2f}")
        print(f"RMSE: {rmse:.2f}")
        print(f"MAPE ajustado: {mape:.2f}%")

        # Armazena os resultados
        resultados.append({
            'Modelo': 'LSTM',
            'Setor': setor,
            'Look_back': look_back,
            'Epochs': epochs,
            'Batch_size': batch_size,
            'MAE': round(mae, 2),
            'RMSE': round(rmse, 2),
            'MAPE (%)': round(mape, 2)
        })

# ✅ Criar DataFrame com todos os resultados
df_resultados_lstm = pd.DataFrame(resultados)



🚩 Iniciando treinamento para setor: BSF

🛠️  Parâmetros: look_back=6, epochs=100, batch_size=16


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 222ms/step
📉 Métricas de avaliação para setor BSF:
MAE: 11.20
RMSE: 15.43
MAPE ajustado: 208.70%

🛠️  Parâmetros: look_back=6, epochs=100, batch_size=32


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 198ms/step
📉 Métricas de avaliação para setor BSF:
MAE: 12.76
RMSE: 17.64
MAPE ajustado: 233.39%

🛠️  Parâmetros: look_back=6, epochs=100, batch_size=64


...

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 190ms/step
📉 Métricas de avaliação para setor BSF:
MAE: 7.33
RMSE: 10.82
MAPE ajustado: 56.67%

🛠️  Parâmetros: look_back=18, epochs=300, batch_size=32


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 211ms/step
📉 Métricas de avaliação para setor BSF:
MAE: 10.22
RMSE: 15.21
MAPE ajustado: 178.62%

🛠️  Parâmetros: look_back=18, epochs=300, batch_size=64


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 194ms/step
📉 Métricas de avaliação para setor BSF:
MAE: 8.01
RMSE: 11.74
MAPE ajustado: 94.60%


In [58]:
# ✅ Criar DataFrame com todos os resultados
df_resultados_lstm = pd.DataFrame(resultados)

# Ordenar pelo menor MAPE
df_resultados_lstm = df_resultados_lstm.sort_values(by='MAPE (%)')

# Exibir todos os resultados
display(df_resultados_lstm)

# 💾 Salvar em arquivo CSV
df_resultados_lstm.to_csv('resultados_lstm_completos.csv', index=False)
print("📁 Resultados salvos como 'resultados_lstm_completos.csv'")



,Modelo,Setor,Look_back,Epochs,Batch_size,MAE,RMSE,MAPE (%)
17,LSTM,BSF,12,300,64,7.03,9.68,46.53
24,LSTM,BSF,18,300,16,7.33,10.82,56.67
16,LSTM,BSF,12,300,32,7.59,10.67,57.50
15,LSTM,BSF,12,300,16,8.76,12.80,64.69
21,LSTM,BSF,18,200,16,8.24,11.76,64.75
6,LSTM,BSF,6,300,16,8.94,13.21,64.92
18,LSTM,BSF,18,100,16,7.52,11.55,80.25
12,LSTM,BSF,12,200,16,10.36,15.23,89.60
26,LSTM,BSF,18,300,64,8.01,11.74,94.60
23,LSTM,BSF,18,200,64,8.38,12.41,112.44


📁 Resultados salvos como 'resultados_lstm_completos.csv'
